# Visualization

Plot the accuracy of the model on a map to visually inspect the accuracy outcomes of the model. 

<b> Requirements : </b> 

- `directory` : the directory in which the accuracy outcomes of the model are stored,
- `aux_dir` : the directory in which the cities characteristics are stored,
- `filename` : the name of the file containing the accuracy outcomes of the model.
- `departements` : a list of departements for which the plots will be made


In [ ]:
import pandas as pd
import numpy as np
import os
import geojson
import folium
import json

In [ ]:
# helper function : computes the overlay that will be displayed on the Folium map.

def compute_geojson_overlay(results, communes_list):
    """
    returns a geojson with the coordinates of the cities 
    and some of their properties. 
    
    args : 
    - results : a datafame of results
    - communes_list : a list of json files corresponding to the cities of the targeted departements.
    """
    
    # retrieve the polygons of the commune

    target_communes = np.unique(results['city'].values)

    features = []

    for communes in communes_list:
        for commune in communes.keys():
            for target in target_communes:

                    if int(target) == int(communes[commune]['properties']['code_insee']):

                        polygon = communes[commune]['coordinates']

                        polygon = geojson.Polygon(polygon)  

                        ape = results[results['city'] == target]['APE'].values.item()

                        properties = {
                            'city' : int(target),
                            'APE' : ape,
                            'Nom' : communes[commune]['properties']['nom_commune']
                        }

                        features.append(geojson.Feature(geometry=polygon, properties=properties))

    feature_collection = geojson.FeatureCollection(features)
    
    return feature_collection

In [ ]:
# directories
directory = 'evaluation/building'
aux_dir = 'aux'
filename = "results_buildings.csv"


# departements : target departements to be plotted
departements = [69, 34, 44, 59]
# center location : the center of the map to be displayed
center = [45.818701443200275, 4.804099378122765]

In [ ]:
# load the files and prepare the communes_list

# results dataframe
results = pd.read_csv(os.path.join(directory, filename))

# communes coordinates
# retrieve the 
communes_list = []

for dpt in departements:

    communes = json.load(open(os.path.join(aux_dir, 'communes_{}.json'.format(dpt))))
    communes_list.append(communes)
    
# compute the geojson that will be displayed
feature_collection = compute_geojson_overlay(results, communes_list)    
        
results.head()

## Example : plot the city-wise APE

In [ ]:

# bins : to be manually set up for a better display.
bins = [0, 15, 75, 100, 150, int(results['APE'].max()) + 1] 

m = folium.Map(location=center, zoom_start=9)

folium.Choropleth(
    geo_data = feature_collection,#geojson.load(open("geodata_results.geojson")),                  #json
    name ='APE',  
    bins = bins,
    data = results,                     
    columns = ['city', 'APE'], #columns to work on
    key_on ='properties.city',
    fill_color ='YlOrBr',     #I passed colors Yellow,Green,Blue
    fill_opacity = 0.7,
    line_opacity = 0.2,
legend_name = "APE"
).add_to(m)

style_dict = {
        "fillColor": "#ffff00",
        "weight": 0,
    }
# add an overlay with the commune properties (name, postal code and APE value)
geo_json = folium.GeoJson(data=feature_collection, popup=folium.GeoJsonPopup(fields=['Nom', 'city', 'APE']), style_function= lambda feature: style_dict)
geo_json.add_to(m)



m

In [ ]:
# identifier une ville où il y a un bon résultat et une autre un mauvais, qui sont proches l'une de l'autre (cas pathologique)
# insister sur le motif géographique et la représentativité